# Timed Petri Net Simulation

A Python implementation of a deterministic, timed Petri net with support for burst firing transitions.

## Table of Contents
1. [Classes](#classes)
   - [Place](#place)
   - [CombinedTransition](#combinedtransition)
   - [PetriNet](#petrinet)
2. [Example Usage](#example-usage)
3. [Key Features](#key-features)

## Classes

### Place
Represents a place in the Petri net that holds tokens.

#### Attributes
- `name` (str): Identifier for the place
- `tokens` (int): Number of tokens currently in the place

#### Methods
- `add_tokens(count: int)`: Adds tokens to the place
- `remove_tokens(count: int)`: Removes tokens from the place (raises ValueError if insufficient tokens)
- `__repr__()`: String representation of the place

### CombinedTransition
Represents a timed transition that can fire in bursts.

#### Attributes
- `name` (str): Transition identifier
- `input_places` (Dict[Place, int]): Input places and their required token weights
- `output_places` (Dict[Place, int]): Output places and their token production weights
- `fixed_delay` (float): Time delay between firings
- `burst_size` (int): Number of firings per burst (default=1)
- `deterministic` (bool): Whether to use fixed delays (default=True)
- `force_schedule` (bool): Always schedule regardless of enabling (default=False)
- `poll_only` (bool): Only check enabling condition when firing (default=False)
- `burst_remaining` (int): Tracks remaining firings in current burst

#### Methods
- `enabling_threshold()`: Returns required tokens for burst firing
- `is_enabled()`: Checks if transition can fire
- `can_actually_fire()`: Verifies token availability for poll_only transitions
- `start_burst()`: Initiates a new burst sequence
- `fire()`: Executes one firing event
- `scheduled_time(current_time: float)`: Calculates next firing time

### PetriNet
The simulation engine that manages places, transitions, and event scheduling.

#### Attributes
- `places` (List[Place]): All places in the net
- `transitions` (List[CombinedTransition]): All transitions in the net
- `current_time` (float): Simulation clock
- `event_queue` (List): Priority queue of scheduled events
- `event_counter` (int): Tie-breaker for simultaneous events

#### Methods
- `schedule_transition()`: Adds an event to the queue
- `initialize_events()`: Sets up initial transition schedules
- `simulate(max_time: float)`: Runs the simulation until max_time
- `print_places()`: Displays current state of all places

## Example Usage

```python
# Create places
P_Ready = Place("P_Ready", 3)  # 3 tasks waiting
P_CPU = Place("P_CPU", 1)      # CPU available
P_Done = Place("P_Done", 0)    # Completed tasks

# Create transitions
T_Dispatch = CombinedTransition(
    name="T_Dispatch",
    input_places={P_Ready: 1, P_CPU: 1},
    output_places={P_CPU: 1},
    fixed_delay=1.0
)

T_Execute = CombinedTransition(
    name="T_Execute",
    input_places={P_CPU: 1},
    output_places={P_Done: 1, P_CPU: 1},
    fixed_delay=10.0
)

# Create and run simulation
net = PetriNet(
    places=[P_Ready, P_CPU, P_Done],
    transitions=[T_Dispatch, T_Execute]
)
net.simulate(max_time=50.0)